In [1]:
import pandas as pd

In [2]:
import nltk
from nltk.tag import StanfordPOSTagger

In [105]:
test = pd.read_csv('gap-coreference-master/gap-validation.tsv', sep = '\t')

In [106]:
test.head(5)

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,validation-1,He admitted making four trips to China and pla...,him,256,Jose de Venecia Jr,208,False,Abalos,241,False,http://en.wikipedia.org/wiki/Commission_on_Ele...
1,validation-2,"Kathleen Nott was born in Camberwell, London. ...",She,185,Ellen,110,False,Kathleen,150,True,http://en.wikipedia.org/wiki/Kathleen_Nott
2,validation-3,"When she returns to her hotel room, a Liberian...",his,435,Jason Scott Lee,383,False,Danny,406,True,http://en.wikipedia.org/wiki/Hawaii_Five-0_(20...
3,validation-4,"On 19 March 2007, during a campaign appearance...",he,333,Reucassel,300,True,Debnam,325,False,http://en.wikipedia.org/wiki/Craig_Reucassel
4,validation-5,"By this time, Karen Blixen had separated from ...",she,427,Finch Hatton,290,False,Beryl Markham,328,True,http://en.wikipedia.org/wiki/Denys_Finch_Hatton


# change name

In [107]:
# change 'pronoun' column in text to 'placeholder'
test['Text'] = test.apply(lambda x: x['Text'][0:x['Pronoun-offset']] +' placeholder '+ \
                          x['Text'][(x['Pronoun-offset']+len(x['Pronoun'])):], axis=1)

In [108]:
# change name to 'Bob', 'Jim', 'Susan' and 'Jane'
def change_name(df):
    if df['Pronoun'] in ['He', 'His','he','his','him']:
        return df['Text'].replace(df['A'], ' Bob ').replace(df['B'], ' Jim ')
    else:
        return df['Text'].replace(df['A'], ' Susan ').replace(df['B'], ' Jane ')
test['Text'] = test.apply(lambda x: change_name(x),axis=1)

def change_A(df):
    if df['Pronoun'] in ['He', 'His','he','his','him']:
        return 'Bob'
    else:
        return 'Susan'
test['A'] = test.apply(lambda x: change_A(x), axis=1)

def change_B(df):
    if df['Pronoun'] in ['He', 'His','he','his','him']:
        return 'Jim'
    else:
        return 'Jane'
test['B'] = test.apply(lambda x: change_B(x), axis=1)

In [109]:
test.iloc[1,1]

" Jane  Nott was born in Camberwell, London. Her father, Philip, was a lithographic printer, and her mother,  Susan , ran a boarding house in Brixton;  Jane  was their third daughter.  placeholder  was educated at Mary Datchelor Girls' School (now closed), London, before attending King's College, London."

In [110]:
# tokenize and get index of pronoun, A, B
test['token_list'] = test['Text'].apply(lambda x: nltk.word_tokenize(x))
test['Pronoun-index'] = test['token_list'].apply(lambda x: x.index('placeholder'))
test['A-index'] = test.apply(lambda x: x['token_list'].index(x['A']) if x['A'] in x['token_list'] else 0, axis=1)
test['B-index'] = test.apply(lambda x: x['token_list'].index(x['B']) if x['B'] in x['token_list'] else 0, axis=1)

In [111]:
# change placeholder to original pronoun
test['token_list'] = test.apply(lambda x: [x['Pronoun'] if i == 'placeholder' else i for i in x['token_list']],axis=1)
test['token_list'] = test['token_list'].apply(lambda x: ' '.join(x) )

In [112]:
with open('gap-coreference-master/name_changed/gap-validation-new.tsv', 'w') as write_tsv:
    write_tsv.write(test.to_csv(sep='\t', index=False))

In [94]:
test.columns

Index(['ID', 'Text', 'Pronoun', 'Pronoun-offset', 'A', 'A-offset', 'A-coref',
       'B', 'B-offset', 'B-coref', 'URL', 'token_list', 'Pronoun-index',
       'A-index', 'B-index'],
      dtype='object')

# change sex

In [95]:
test = pd.read_csv('gap-coreference-master/name_changed/gap-test-new.tsv', sep = '\t')

In [84]:
test.groupby(['Pronoun']).count()

,ID,Text,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,token_list,Pronoun-index,A-index,B-index
Pronoun,,,,,,,,,,,,,,
He,127,127,127,127,127,127,127,127,127,127,127,127,127,127
Her,37,37,37,37,37,37,37,37,37,37,37,37,37,37
His,28,28,28,28,28,28,28,28,28,28,28,28,28,28
She,159,159,159,159,159,159,159,159,159,159,159,159,159,159
he,221,221,221,221,221,221,221,221,221,221,221,221,221,221
her,566,566,566,566,566,566,566,566,566,566,566,566,566,566
hers,1,1,1,1,1,1,1,1,1,1,1,1,1,1
him,96,96,96,96,96,96,96,96,96,96,96,96,96,96
his,528,528,528,528,528,528,528,528,528,528,528,528,528,528


In [113]:
test['token_list'] = test['token_list'].apply(lambda x: x.split(' '))

In [114]:
test.iloc[1,11]

['Jane',
 'Nott',
 'was',
 'born',
 'in',
 'Camberwell',
 ',',
 'London',
 '.',
 'Her',
 'father',
 ',',
 'Philip',
 ',',
 'was',
 'a',
 'lithographic',
 'printer',
 ',',
 'and',
 'her',
 'mother',
 ',',
 'Susan',
 ',',
 'ran',
 'a',
 'boarding',
 'house',
 'in',
 'Brixton',
 ';',
 'Jane',
 'was',
 'their',
 'third',
 'daughter',
 '.',
 'She',
 'was',
 'educated',
 'at',
 'Mary',
 'Datchelor',
 'Girls',
 "'",
 'School',
 '(',
 'now',
 'closed',
 ')',
 ',',
 'London',
 ',',
 'before',
 'attending',
 'King',
 "'s",
 'College',
 ',',
 'London',
 '.']

In [115]:
# change 'her' to 'her1' and 'her2'
def check_her(tokens):
    model = 'gap-coreference-master/english-bidirectional-distsim.tagger'
    jar = 'gap-coreference-master/stanford-postagger.jar'
    st = StanfordPOSTagger(model,path_to_jar=jar)
    t = st.tag(tokens)
    # her1 宾语 her2 所有格
    for idx, word in enumerate(tokens):
        if word == 'her':
            tokens[idx] = 'her1' if t[idx][1] == 'PRP' else 'her2'
    return tokens

test['token_list'] = test['token_list'].apply(lambda x: check_her(x))

In [121]:
test.iloc[9,11]

['They',
 'have',
 'a',
 'stormy',
 'marriage',
 ',',
 'caused',
 'by',
 'his',
 'hot',
 'temper',
 'and',
 'her2',
 'habit',
 'of',
 'playing',
 'away',
 '.',
 'Soon',
 'after',
 'arriving',
 'on',
 'the',
 'Street',
 ',',
 'Jane',
 'makes',
 'friends',
 'with',
 'Deirdre',
 'Barlow',
 '(',
 'Anne',
 'Kirkbride',
 ')',
 ',',
 'whose',
 'daughter',
 'Tracy',
 '(',
 'Dawn',
 'Acton',
 ')',
 'is',
 'slightly',
 'younger',
 'than',
 'the',
 'twins',
 '.',
 'When',
 'asked',
 'in',
 'a',
 '2010',
 'interview',
 'with',
 'The',
 'Mirror',
 'what',
 'her2',
 'favourite',
 'scenes',
 'were',
 ',',
 'Susan',
 'replied',
 ',',
 '``',
 'when',
 'Jim',
 'beat',
 'up',
 'Jane',
 '.']

In [130]:
# change sex
def replace_sex(tokens):
    sentence = ' '.join(tokens)
    sentence = sentence.replace('She','He1').replace('she','he1').replace('hers','his1').replace('Her','His1').replace('He ','She ').replace('His ','Her ').replace(' he ',' she ').replace(' him ',' her ').replace(' his ' ,' her ')
    sentence = sentence.replace('He1','He').replace('he1','he').replace('his1','his').replace('His1','His').replace('her1','him').replace('her2','his')
    return sentence.split(' ')
test['token_list2'] = test['token_list'].apply(lambda x: replace_sex(x))

In [140]:
# change pronoun sex
test['Pronoun'] = test.apply(lambda x: x['token_list2'][x['Pronoun-index']], axis=1)

In [ ]:
with open('gap-coreference-master/gap-validation-new.tsv', 'w') as write_tsv:
    write_tsv.write(test.to_csv(sep='\t', index=False))